In [1]:
import os
import pandas as pd
import numpy as np
import re
from scipy.signal import argrelmin, argrelmax
from bisect import bisect_right

#Plotting
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = (15, 10)
import seaborn as sns
pd.options.mode.chained_assignment = None

In [8]:
def processData(game_id):
    os.chdir("C:\Users\ckchiruka\Desktop\Current Courses\STA 160\CSV Data")
    motion=pd.read_csv("00"+str(game_id)+".csv",low_memory=False)
    motion=motion[motion.isnull().any(axis=1)==False].reset_index(drop=True)
    return(motion)

def getPlayRanges(motion):
    start = 0
    end = 0
    inplay = []
    no_dups = motion.drop_duplicates(subset = ['quarter', 'sec_in_quarter', 'shot_clock'], keep = 'last')
    nd_index = no_dups.index
    for i in range(max(nd_index)):
        if i not in nd_index:
            start = i
        else:
            inplay.append(start+1)
    
        if i in nd_index:
            end = i  
        else:
            inplay.append(end)

    inplay = sorted(list(set(inplay)))
    inplay = [i for i in inplay if i + 1 in nd_index or i - 1 in nd_index]

    if len(inplay) % 2 == 1:
        inplay.append(len(motion)-1)
    
    inplay = np.asarray(inplay).reshape((len(inplay)/2,2))

    minima = getRelMinima(motion)
    
    for i in range(len(inplay)):
        try:
            inplay[i][0] = minima[bisect_right(minima, inplay[i][0])-1]
        except IndexError:
            pass
        try: 
            inplay[i][1] = minima[bisect_right(minima, inplay[i][1])]
        except IndexError:
            pass
    return(inplay)

def getRelMinima(motion):
    minima = list(argrelmin(motion['ball_z'].rolling(4, center = True).mean().as_matrix()))
    minima = [i for i in minima[0]]
    return minima

def getRelMaxima(motion):
    maxima = list(argrelmax(motion['ball_z'].rolling(4, center = True).mean().as_matrix()))
    maxima = [i for i in maxima[0]]
    return(maxima)

In [12]:
motion = processData(21500592)b  
ranges = getPlayRanges(motion)